In [ ]:
#Importación de las librerías usadas en el ETL
import pandas as pd
import numpy as np
import ast
from datetime import date

In [ ]:
#Carga de dataset
df_movies = pd.read_csv('C:\Python\HENRY-Data-Science\Proyecto_individual_I\Repositorio-proyecto-I\Proy_uno\Datasets\movies_dataset.csv')
df_credits = pd.read_csv('C:\Python\HENRY-Data-Science\Proyecto_individual_I\Repositorio-proyecto-I\Proy_uno\Datasets\credits.csv')

In [ ]:
#Exploración de columnas he información de los datasets
df_movies.head()

In [ ]:
#Se exploran las columnas con estructura de diccionario para determinar la información que almacena para discriminar
#si tienen información util para el proyecto
col = df_movies['belongs_to_collection']
row = col.iloc[0]
dic = ast.literal_eval(row)

df_dict = pd.DataFrame.from_dict(dic, orient='index')
df_dict

In [ ]:
#Se elimina las columnas que son de utilidad para el proyecto
del_cols = ['adult', 'belongs_to_collection', 'homepage', 'imdb_id', 'original_language', 'original_title', 'overview', 'poster_path',
'production_companies', 'production_countries', 'runtime', 'spoken_languages', 'tagline', 'video']

df_movies.drop(labels=del_cols, axis=1, inplace=True)
df_movies.head()

In [ ]:
#Se verifica si existen valores nulos en las columnas 'budget' y 'revenue'
print(df_movies['revenue'].isnull().sum())
print(df_movies['budget'].isnull().sum())

In [ ]:
df_movies['revenue'].fillna(value=0, inplace=True) #se reemplazan los valores nulos encontrados por 0

In [ ]:
#Transformación de columna 'release-date' en una clase datetime y 
df_movies['release_date'] = pd.to_datetime(df_movies['release_date'], errors='coerce')
df_movies.dropna(subset=['release_date'], inplace=True) #Se eliminan los valores nulos de la columna 'release_date'

In [ ]:
#Se agrega la columna del año
df_movies['year'] = df_movies['release_date'].dt.year.astype('Int64')

In [ ]:
#Se transforma la columna 'budget' a formato numérico y se eliminan los errores
df_movies['budget'] = pd.to_numeric(df_movies['budget'], downcast='signed', errors='coerce')
df_movies.dropna(subset=['budget'], inplace=True)

In [ ]:
#Se crea la columna 'return'
df_movies['return'] = df_movies['revenue']/df_movies['budget']
df_movies['return'] = df_movies['return'].replace([np.nan, np.inf], 0) #reemplazo con 0 los valores que presentan errores

In [ ]:
#Desanidación de columna 'genre'
df_generos = pd.DataFrame(columns=['genres'])   #Se crea df auxiliar
for index_1, row_1 in df_movies.iterrows():     #Se itera sobre la columna 'genre' con la información anidada
    x = ast.literal_eval(row_1['genres'])       #Se transforma a un objeto lista
    df_dict = pd.DataFrame.from_dict(x)         #Se crea un df
    
    ls = []
    
    for index_2, row_2 in df_dict.iterrows():   #Se itera sobre el df_dict para extraer todos los datos de la columna 'name' y se agregan a la lista
        ls.append(row_2['name'])
    
    df_generos.loc[index_1, 'genres'] = ls      #Se agrega la información al df auxiliar

In [ ]:
#Se reemplazan las columnas en el df original
df_movies.drop(labels='genres', axis=1, inplace=True)
df_movies = pd.concat([df_movies, df_generos], axis=1)